In [1]:
from collections import defaultdict
import os
import numpy as np
from sklearn.cluster import KMeans

In [2]:
names = list(set([fname.split('.')[0] for fname in os.listdir('D:\\mk\\np_data\\')]))
recovered = defaultdict(list)
#slovar monophone i ih features
for name in names:
#Po4emu-to machina umiraet esli odnovremenno np.load targs i feats
    targs = np.load('D:\\mk\\np_data\\'+name+'.targs.npy')
    feats = np.load('D:\\mk\\np_data\\'+name+'.feats.npy')
    for i,j in enumerate(targs):
        recovered[j] += [feats[i]]
for i in recovered:
    recovered[i] = np.array(recovered[i])

In [44]:
scanned = {}
with open('D:\\mk\\data\\lang\\phones.txt') as phns:
    for i in phns:
        scanned[int(i.split()[1])] = i.split()[0].split('_')[0]
scanned[6112] = 'STRT'
scanned[6119] = 'END'
    
    
    
    #    for i in phns:
        
#        scanned[int(i.split()[1])] = i.split('_')[0]

In [9]:
def create_triphones(given):
    
    # A GIANT MESS, VERY SORRY ABOUT THAT. PRAVDA
    
    phnm =[6112] + given.tolist() + [6119] #otmetit' na4alo i konec predlojenija

    cash = []
    counter = 1
    safe1 = 1
    counter2 = 0
    for i in range(1, len(phnm)-1):
        if phnm[i] != phnm[i-1]:
            if counter2 > 0:                        
                for ne_i_to4no in range(counter):
                    cash.append((phnm[safe1-1],phnm[safe1],phnm[i]))
            safe1 = i
            counter2 += 1
            counter = 1
        else:
            counter += 1
    for ne_i_to4no in range(counter):
        cash.append((phnm[safe1-1],phnm[safe1],phnm[-1]))
    return cash

In [10]:
#slovar triphone i feature
triphone = defaultdict(list)
for name in names:
#Po4emu-to machina umiraet esli odnovremenno np.load targs i feats
    targs = create_triphones(np.load('D:\\mk\\np_data\\'+name+'.targs.npy'))
    feats = np.load('D:\\mk\\np_data\\'+name+'.feats.npy')
    for i,j in enumerate(targs):
        triphone[j] += [feats[i]]


In [90]:
def mu_sigma(feats):
    mean = np.mean(feats, axis=0)
    std = np.std(feats, axis=0)
    return mean, std
def check_params(tri):
    tri_means, tri_sigmas = mu_sigma(triphone[tri])
    return tri_means, tri_sigmas

можно посмотреть параметры любых трифонов

In [78]:
class kmean_holder():
    def __init__(self, triphone_number, triphones = triphone, k = 4):

        brotherhood = []

        for tri in triphones:
            if tri[1] == triphone_number:
                brotherhood.append(tri)

        matrix = []

        for brother in brotherhood:
            matrix.append(np.hstack((np.mean(brother, axis=0),np.std(brother, axis=0))))
        kmeans = KMeans(n_clusters=k, random_state=0).fit(matrix)
        self.kmean = kmeans
        clusters = defaultdict(list)
        self.brotherhood = brotherhood
        for i, cl in enumerate(kmeans.labels_):
            clusters[int(cl)+1] += [[scanned[brother_part] for brother_part in brotherhood[i]]]
        self.clusters = clusters
        
        
    
    

Я совсем не лингвист, поэтому мне весьма затруднительно произвести анализ получившихся кластеров. Их 4, потому что это, как правило, всегда дешево и сердито. Кластер 4 получился маленьким, но из-за того это, что кластеризация вышла неудачной, или потому, что это действительно выделяющийся класс, по причине отсутствия лингвистической квалификации/невозможности (ну, скорее, не очень охотном желании, конечно) взглянуть на данные глазами в пространстве я точно не могу. Вроде бы визуально кластер 3 и 4 ближе друг к другу, чем кластер 1 и 2 по набору начальных звуков, например. Вроде нормально. 

In [85]:
z = kmean_holder(42,k = 4)
z.clusters

defaultdict(list,
            {1: [['S', 'AO1', 'NG'],
              ['L', 'AO1', 'R'],
              ['P', 'AO1', 'R'],
              ['IY0', 'AO1', 'R'],
              ['K', 'AO1', 'R'],
              ['F', 'AO1', 'T'],
              ['K', 'AO1', 'SH'],
              ['R', 'AO1', 'L'],
              ['S', 'AO1', 'N'],
              ['HH', 'AO1', 'T'],
              ['R', 'AO1', 'M'],
              ['T', 'AO1', 'K'],
              ['K', 'AO1', 'T'],
              ['L', 'AO1', 'S'],
              ['P', 'AO1', 'N'],
              ['P', 'AO1', 'L'],
              ['W', 'AO1', 'F'],
              ['P', 'AO1', 'NG'],
              ['N', 'AO1', 'R'],
              ['K', 'AO1', 'R'],
              ['JH', 'AO1', 'R'],
              ['S', 'AO1', 'NG'],
              ['L', 'AO1', 'R'],
              ['P', 'AO1', 'R'],
              ['S', 'AO1', 'L'],
              ['D', 'AO1', 'Z'],
              ['N', 'AO1', 'R'],
              ['R', 'AO1', 'NG'],
              ['T', 'AO1', 'L'],
             